# Upload data to S3

In [14]:
import sagemaker
import pathlib
import boto3

In [42]:
BASE_DIR = pathlib.Path().resolve().parent

DATA_DIR = BASE_DIR / 'data'

EXPORTS_DIR = DATA_DIR / 'exports'
XRAY_LUNG_CLF_DIR = DATA_DIR / 'xray_lung_clf'

TRAIN_DATA_PATH = XRAY_LUNG_CLF_DIR / 'train'
TEST_DATA_PATH = XRAY_LUNG_CLF_DIR / 'test'

TRAIN_ANNOTATIONS_PATH = XRAY_LUNG_CLF_DIR / 'train_annotations.json'
TEST_ANNOTATIONS_PATH = XRAY_LUNG_CLF_DIR / 'test_annotations.json'

EXPORTS_LUNG_CLF_DIR = EXPORTS_DIR / 'xray_lung_clf'

serve_pytorch_dir = BASE_DIR / 'serve_pytorch'
train_py_path = serve_pytorch_dir / 'train.py'

print(train_py_path)

/home/ec2-user/SageMaker/UdacityCapstone/serve_pytorch/train.py


In [11]:
# Create session
session = sagemaker.Session()

# Get Role
role = sagemaker.get_execution_role()

# Get Bucket
bucket = session.default_bucket()

In [19]:
# Get data dir
data_dir = str(EXPORTS_LUNG_CLF_DIR)

# set prefix, a descriptive name for a directory  
prefix = 'lung_data'

# upload all data to S3
input_data = session.upload_data(data_dir, bucket=bucket, key_prefix=prefix)
print(input_data)

s3://sagemaker-us-east-1-906764816799/lung_data


In [20]:
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    print(obj.key)

lung_data/test_X.csv
lung_data/test_Y.csv
lung_data/train_X.csv
lung_data/train_Y.csv


# Create PyTorch esitmator

In [55]:
path = str(train_py_path.name)
print(path)

train.py


In [53]:
# directory can be changed to: source_sklearn or source_pytorch
!pygmentize $path

import argparse
import json
import os
import pandas as pd
import torch
import torch.optim as optim
import torch.utils.data

# imports the model in model.py by name
from model import VisionTransformer

def model_fn(model_dir):
    """Load the PyTorch model from the `model_dir` directory."""
    print("Loading model.")

    # First, load the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    # Determine the device and construct the model.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = VisionTransformer(image_size=256, patch_size=32, num_classes=13, channels=1,
                               k=64, depth=3, heads=8, mlp_dim=64)

    # Load the stored model parameters.
    model_path = os.path.join(model_dir, 'model.pth')
    with open(model_path, 'rb') a

In [57]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
                    entry_point = 'train.py',
                    source_dir = str(serve_pytorch_dir),
                    role = role,
                    instance_count = 1,
                    instance_type = 'ml.g4dn.xlarge',
                    framework_version='1.8.0',
                    py_version='py3',
                    sagemaker_session = session)

estimator.fit(input_data)

2021-11-29 19:52:42 Starting - Starting the training job...
2021-11-29 19:53:11 Starting - Launching requested ML instancesProfilerReport-1638215562: InProgress
...
2021-11-29 19:53:42 Starting - Insufficient capacity error from EC2 while launching instances, retrying!...
2021-11-29 19:54:12 Failed - Training job failed
..

UnexpectedStatusException: Error for Training job pytorch-training-2021-11-29-19-52-42-536: Failed. Reason: CapacityError: Unable to provision requested ML compute capacity. Please retry using a different ML instance type.

# Deploy Estimator